In [1]:
import csv
import tweepy
import pandas as pd
import spacy
from spacy.lang.id import Indonesian
import unicodedata
from unidecode import unidecode
import html
import re
import emoji
from googletrans import Translator
import json

import nltk
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.stem import WordNetLemmatizer
# from nltk.stem import PorterStemmer
from nltk.util import ngrams
from nltk.corpus import sentiwordnet as swn

from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import neighbors
from sklearn.metrics import accuracy_score

C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\USER\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# Auth
auth = tweepy.OAuthHandler('ynr485xkIGMh5m3RwEhhiK3Y4', 'xooRDYC235j8FTkeZe3s3NcmvzMqndy264RXAVwIQvvZsksbGk')
auth.set_access_token('1226612648-kimIpngGwgx3tceLibaIjNpVUAyfjLm2DjkwVa5', 'hknzO1NV885ghvSsymN5ihJArAqj1tLZZXlitXrJcM1ke')

api = tweepy.API(auth)

In [5]:
# Crawl
venom_jkt_tweets = tweepy.Cursor(api.search, q='fantastic beast', geocode="-6.265750,106.781821,150km").items()

In [20]:
#CSV File
with open('fb_jkt_tweetss.csv', 'w', encoding="utf-8") as csv_tweet:

    fieldnames_tweet = ['id', 'user_id', 'created_at', 'text']
    writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
    writer_tweet.writeheader()

    for tweet in venom_jkt_tweets:
        if "http" not in tweet.text and "@" not in tweet.text:
            #line = re.sub("[^A-Za-z]", " ", tweet.text)
            #user_id = tweet.user.id
            writer_tweet.writerow({
                'id': tweet.id, 
                'user_id': tweet.user.screen_name,
                'created_at': tweet.created_at, 
                'text': tweet.text
            })      
        

In [46]:
#load dataset

fb_jkt = pd.read_csv('fb_jkt_tweets.csv')
# fb_text = fb_jkt['text']
fb_jkt.head()

,id,user_id,created_at,text
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :')
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa"
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter..."


In [33]:
#Preprocessing

nlp_spacy_id = Indonesian()
pd.set_option('max_colwidth', 100)

def normalize_slang_words(tokenized_sentence):
    slang_word_dict = json.loads(open('slang_word_dict.txt', 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

def normalize_repeated_char(tokenized_sentence):
    alphabet = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
                'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    tokenized_sentence = ' '.join(tokenized_sentence)
    for i in range(len(alphabet)):
        charac_long = 5
        while charac_long >= 3:
            char = alphabet[i] * charac_long
            tokenized_sentence = tokenized_sentence.replace(char, alphabet[i])
            charac_long -= 1        
    tokenized_sentence = tokenized_sentence.split()    
    return tokenized_sentence

def preprocessing(comment):
    
    comment = comment.lower() #lower
    ContentnoHTML = html.unescape(str(comment)) #html tag 
    tokenS_id = nlp_spacy_id(ContentnoHTML) #Token
    ContentnoRepeat = normalize_repeated_char(token.text for token in tokenS_id) #repeated char
    ContentnoSlang = normalize_slang_words(ContentnoRepeat) #Slang
    ContentnoEmoji = emoji.demojize(str(ContentnoSlang)) #Emoji
    ContentnoPunct = re.sub('[^ a-zA-Z0-9]', ' ',ContentnoEmoji) #Punctuation
    
#     stopfactory = StopWordRemoverFactory() #Stop word
#     stopword = stopfactory.create_stop_word_remover()
#     stopRemove = stopword.remove(ContentnoPunct)

    content = list()
    content.append(ContentnoPunct)
 
    return " ".join(content)

fb_jkt['Clean'] = fb_jkt['text'].apply(preprocessing)
fb_jkt.head(20)

,id,user_id,created_at,text,Clean,Translate
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :'),begitu gajian rasanya ingin nonton fantastic beast suzanna w...,once the payday feels like watching fantastic beast and wrapping it instantly a day
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa",saran film yang seru model fantastic beast kalau bisa,exciting movie advice on a fantastic beast if possible
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast terus paan sih ini,"after watching fantastic beast, then this guy"
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...,habis nonton fantastic beast terus langsung open discussion dibah...,"after watching fantastic beast, the open discussion was immediately discussed until Malem lost m..."
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter...",baru sempat nonton fantastic beast satu pikiran saya daritadi ...,"I had just watched my fantastic beast one thought from this, so my curiosity had to be answered ..."
5,1065986839171878912,annisa_rianti,2018-11-23 15:14:02,Fix pemeran Fantastic Beast udah dipantau sejak lama sama tim. Callum Turner pernah main sama Da...,fix pemeran fantastic beast sudah dipantau sejak lama sama tim ...,the fix for the fantastic beast cast has been monitored for a long time with the callum turner t...
6,1065984642497441793,shittymuc,2018-11-23 15:05:18,Fantastic Beast 2 didn't look so fantastic like the 1st one. But still loved Eddie ❤️ #fridaynig...,fantastic beast 2 didn t look so fantastic like the 1st one ...,fantastic beast 2 didn t look so fantastic like the 1st one ...
7,1065970037813432321,abrarkadabra,2018-11-23 14:07:16,Ralph breaks the internet &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; Fantastic Beast : Rise of the silver ...,ralph breaks the internet fantastic bea...,ralph breaks the internet gt fantastic beast rise ...
8,1065969011962216450,ineedspacexx,2018-11-23 14:03:11,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gimana.. nunggu bertahun tahun...,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gima...,"after watching fantastic beast, I was curious about the follow-up, how about waiting for more years"
9,1065946976129212416,AristaRiris1,2018-11-23 12:35:38,"Ralph udh main hari ini, belum nonton Fantastic beast.. belum nonton Yg lain lain lain :((( 😣",ralph sudah main hari ini belum nonton fantastic beast b...,Ralph has been playing today but hasn't watched Fantastic Beast yet watching other others persev...


In [34]:
#translate

def translating(indo_blob):
    translator = Translator()
    
    blob = translator.translate(indo_blob, dest='en').text 
    contentTrans = list()
    contentTrans.append(blob)
    
    return " ".join(contentTrans)

fb_jkt['Translate'] = fb_jkt['Clean'].apply(translating)
fb_jkt.head(20)

,id,user_id,created_at,text,Clean,Translate
0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :'),begitu gajian rasanya ingin nonton fantastic beast suzanna w...,once the payday feels like watching fantastic beast and wrapping it instantly a day
1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa",saran film yang seru model fantastic beast kalau bisa,exciting movie advice on a fantastic beast if possible
2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast terus paan sih ini,"after watching fantastic beast, then this guy"
3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...,habis nonton fantastic beast terus langsung open discussion dibah...,"after watching fantastic beast, the open discussion was immediately discussed until Malem lost m..."
4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter...",baru sempat nonton fantastic beast satu pikiran saya daritadi ...,"I had just watched my fantastic beast one thought from this, so my curiosity had to be answered ..."
5,1065986839171878912,annisa_rianti,2018-11-23 15:14:02,Fix pemeran Fantastic Beast udah dipantau sejak lama sama tim. Callum Turner pernah main sama Da...,fix pemeran fantastic beast sudah dipantau sejak lama sama tim ...,the fix for the fantastic beast cast has been monitored for a long time with the callum turner t...
6,1065984642497441793,shittymuc,2018-11-23 15:05:18,Fantastic Beast 2 didn't look so fantastic like the 1st one. But still loved Eddie ❤️ #fridaynig...,fantastic beast 2 didn t look so fantastic like the 1st one ...,fantastic beast 2 didn t look so fantastic like the 1st one ...
7,1065970037813432321,abrarkadabra,2018-11-23 14:07:16,Ralph breaks the internet &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; Fantastic Beast : Rise of the silver ...,ralph breaks the internet fantastic bea...,ralph breaks the internet fantastic beast...
8,1065969011962216450,ineedspacexx,2018-11-23 14:03:11,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gimana.. nunggu bertahun tahun...,setelah nonton fantastic beast saya penasaran nanti kelanjutannya gima...,"after watching fantastic beast, I was curious about the follow-up, how about waiting for more years"
9,1065946976129212416,AristaRiris1,2018-11-23 12:35:38,"Ralph udh main hari ini, belum nonton Fantastic beast.. belum nonton Yg lain lain lain :((( 😣",ralph sudah main hari ini belum nonton fantastic beast b...,Ralph has been playing today but hasn't watched Fantastic Beast yet watching other others persev...


In [35]:
# save clean data

fb_jkt.to_csv('fb_jkt_Clean.csv', sep=',',index=True, encoding='utf-8')

In [3]:
#load dataset clean

fb_jkt_Clean = pd.read_csv('fb_jkt_Clean.csv')
# fb_text = fb_jkt['text']
fb_jkt_Clean.head()

,Unnamed: 0,id,user_id,created_at,text,Clean,Translate
0,0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic ...,begitu gajian rasanya ingin nont...,once the payday feels like watching fantastic ...
1,1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kal...",saran film yang seru model ...,exciting movie advice on a fantastic beast if ...
2,2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast ter...,"after watching fantastic beast, then this guy"
3,3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung o...,habis nonton fantastic beast ...,"after watching fantastic beast, the open discu..."
4,4,1065989076019376128,gebifelina,2018-11-23 15:22:55,Baru sempet nonton fantastic beast. Satu pikir...,baru sempat nonton fantastic bea...,I had just watched my fantastic beast one thou...


In [4]:
#load dataset clean labeled

fb_jkt_label = pd.read_csv('fb_jkt_Clean_labeled.csv')
# fb_text = fb_jkt['text']
fb_jkt_label.head()

,Unnamed: 0,id,user_id,created_at,text,Clean,Translate,Labels
0,0,1.070000e+18,cessphoebe,11/24/2018 4:57,Begitu gajian rasanya pengen nonton fantastic ...,begitu gajian rasanya ingin nont...,once the payday feels like watching fantastic ...,positif
1,1,1.070000e+18,aulazack,11/23/2018 18:44,"Saran film yg seru? Model"" fantastic beast kal...",saran film yang seru model ...,exciting movie advice on a fantastic beast if ...,positif
2,2,1.070000e+18,setchan89,11/23/2018 16:09,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast ter...,"after watching fantastic beast, then this guy",negatif
3,3,1.070000e+18,niekoap,11/23/2018 15:33,Habis nonton fantastic beast. Terus langsung o...,habis nonton fantastic beast ...,"after watching fantastic beast, the open discu...",positif
4,4,1.070000e+18,gebifelina,11/23/2018 15:22,Baru sempet nonton fantastic beast. Satu pikir...,baru sempat nonton fantastic bea...,I had just watched my fantastic beast one thou...,positif


In [5]:
pos = 0
neg = 0
neutral = 0

for index, row in fb_jkt_label.iterrows():
    if row['Labels'] == 'positif':
        pos +=1
    elif row['Labels'] == 'negatif':
        neg +=1
    elif row['Labels'] == 'neutral':
#         print(row["Clean"])
        neutral +=1
        
print(pos, neg, neutral)

256 123 64


In [6]:
#get value for tweet

def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    
    return [ ' '.join(grams) for grams in n_grams]

fitur = list()

for index, word in fb_jkt_Clean.iterrows():
    translated = word['Translate']

    lemmatizer = WordNetLemmatizer()
    
    #bigrams = get_ngrams(translated, 2)
    unigrams = get_ngrams(translated, 1)
    pos_tags = nltk.pos_tag(unigrams)
    
    #singles_bigram = [stemmer.stem(bigram) for bigram in bigrams]
    singles_unigram = [lemmatizer.lemmatize(unigram) for unigram in unigrams]
    
    
    pos_unigram = 0
    neg_unigram = 0
    neutral_unigram = 0
    for unigram in singles_unigram:
        synsets = wn.synsets(unigram)
        for synset in synsets :
            senti = swn.senti_synset(str(synset)[8:-2])

            pos_unigram += senti.pos_score()
            neg_unigram += senti.neg_score()
            neutral_unigram += senti.obj_score() 
    
    fitur.append([pos_unigram,neg_unigram,neutral_unigram])         
            
fitur

[[7.875, 3.375, 61.75],
 [5.25, 4.0, 25.75],
 [3.125, 1.875, 23.0],
 [8.0, 16.625, 102.375],
 [12.5, 10.25, 154.25],
 [6.875, 5.875, 133.25],
 [15.875, 10.375, 80.75],
 [6.125, 5.25, 111.625],
 [4.125, 3.5, 52.375],
 [7.625, 6.75, 99.625],
 [12.25, 6.0, 138.75],
 [8.625, 12.125, 86.25],
 [7.125, 6.375, 44.5],
 [4.125, 6.125, 103.75],
 [5.125, 3.75, 37.125],
 [5.0, 3.75, 70.25],
 [4.5, 6.0, 55.5],
 [8.625, 6.875, 100.5],
 [6.75, 2.5, 53.75],
 [9.875, 21.75, 189.375],
 [3.25, 3.625, 42.125],
 [12.75, 15.375, 164.875],
 [23.0, 7.875, 141.125],
 [5.875, 2.625, 33.5],
 [3.0, 3.25, 61.75],
 [4.375, 6.875, 20.75],
 [6.75, 7.875, 78.375],
 [6.375, 7.375, 86.25],
 [12.125, 6.625, 95.25],
 [2.5, 3.375, 8.125],
 [7.75, 8.625, 72.625],
 [7.25, 6.125, 27.625],
 [23.125, 4.125, 147.75],
 [5.125, 7.0, 108.875],
 [8.125, 3.0, 37.875],
 [7.125, 8.125, 96.75],
 [10.625, 3.375, 138.0],
 [11.375, 5.25, 96.375],
 [5.375, 4.75, 40.875],
 [6.125, 3.75, 48.125],
 [7.625, 6.75, 68.625],
 [5.625, 5.875, 54.5],


In [47]:
#balancing



,Unnamed: 0,id,user_id,created_at,text,Clean,Translate
0,0,1066193963265085440,cessphoebe,2018-11-24 04:57:04,Begitu gajian rasanya pengen nonton fantastic beast-suzanna-wreck it ralph langsung sehari :'),begitu gajian rasanya ingin nonton fantastic beast suzanna w...,once the payday feels like watching fantastic beast and wrapping it instantly a day
1,1,1066039906655338496,aulazack,2018-11-23 18:44:54,"Saran film yg seru? Model"" fantastic beast kalo bisa",saran film yang seru model fantastic beast kalau bisa,exciting movie advice on a fantastic beast if possible
2,2,1066000870058278912,setchan89,2018-11-23 16:09:47,Abis nonton fantastic beast trus PAAN SIH INI,habis nonton fantastic beast terus paan sih ini,"after watching fantastic beast, then this guy"
3,3,1065991814400770051,niekoap,2018-11-23 15:33:48,Habis nonton fantastic beast. Terus langsung open discussion dibahas sampe malem. Ngalahin disku...,habis nonton fantastic beast terus langsung open discussion dibah...,"after watching fantastic beast, the open discussion was immediately discussed until Malem lost m..."
4,4,1065989076019376128,gebifelina,2018-11-23 15:22:55,"Baru sempet nonton fantastic beast. Satu pikiran gua daritadi "" ini rasa penasaran gua harus ter...",baru sempat nonton fantastic beast satu pikiran saya daritadi ...,"I had just watched my fantastic beast one thought from this, so my curiosity had to be answered ..."


In [10]:
# try K-NN

# spilt data

Labels = np.array(fb_jkt_label['Labels'].values.tolist())


X_train, X_test, Y_train, Y_test = train_test_split(np.array(fitur), Labels, test_size=.3)
print(X_train.shape, X_test.shape)

# Inisiasi klasifikasi

n_neighbors = 20
weights = 'distance'
kNN = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
kNN.fit(X_train, Y_train)
print('Done!')

# Prediksi dengan k-NN
y_kNN = kNN.predict(X_test)

# Akurasi

accuracy_score(Y_test, y_kNN)

(310, 3) (133, 3)
Done!


0.5263157894736842

In [21]:
#bp_surabaya_tweets = tweepy.Cursor(api.search, q='venom', geocode="-7.2574719,112.75208829999997,50km").items()

In [25]:
# with open('bp_surabaya_tweets.csv', 'w', encoding="utf-8") as csv_tweet, open('bp_users_surabaya.csv', 'w', encoding="utf-8") as csv_user:
#     fieldnames_tweet = ['id', 'text', 'user_id', 'created_at']
#     writer_tweet = csv.DictWriter(csv_tweet, fieldnames=fieldnames_tweet)
#     writer_tweet.writeheader()

#     fieldnames_user = ['id', 'name', 'screen_name', 'location', 'followers_count', 'friends_count']
#     writer_user = csv.DictWriter(csv_user, fieldnames=fieldnames_user)
#     writer_user.writeheader()

#     for tweet in bp_surabaya_tweets:
#         if not tweet.retweeted and ('RT @' not in tweet.text):
#             writer_tweet.writerow({
#                 'id': tweet.id, 
#                 'text': tweet.text, 
#                 'user_id': tweet.user.id,
#                 'created_at': tweet.created_at
#             })


#             writer_user.writerow({
#                 'id': tweet.user.id, 
#                 'name': tweet.user.name, 
#                 'screen_name': tweet.user.screen_name,
#                 'location': tweet.user.location,
#                 'followers_count': tweet.user.followers_count,
#                 'friends_count': tweet.user.friends_count
#             })        
        

In [54]:
# auth = tweepy.OAuthHandler('ynr485xkIGMh5m3RwEhhiK3Y4', 'xooRDYC235j8FTkeZe3s3NcmvzMqndy264RXAVwIQvvZsksbGk')
# auth.set_access_token('1226612648-kimIpngGwgx3tceLibaIjNpVUAyfjLm2DjkwVa5', 'hknzO1NV885ghvSsymN5ihJArAqj1tLZZXlitXrJcM1ke')

# api = tweepy.API(auth)

In [58]:
# #Getting Geo ID for USA
# places = api.geo_search(lat="-6.265750",long="106.781821", accuracy="100000m")

# #Copy USA id
# place_id = places[0].id
# print('JKT id is: ',place_id)

JKT id is:  e92102d30403f7cf


In [59]:
# #This is what we are searching for
# #We can restrict the location of tweets using place:id 
# #We can search for multiple phrases using OR
# searchQuery = 'place:e92102d30403f7cf "venom" OR "film venom" OR "nonton venom" OR "nonton film venom"'

In [60]:
# #Maximum number of tweets we want to collect 
# maxTweets = 1000000

# #The twitter Search API allows up to 100 tweets per query
# tweetsPerQry = 100

In [64]:
# tweetCount = 0

# #Open a text file to save the tweets to
# with open('PoGo_USA_Tutorial.json', 'w') as f:

#     #Tell the Cursor method that we want to use the Search API (api.search)
#     #Also tell Cursor our query, and the maximum number of tweets to return
#     for tweet in tweepy.Cursor(api.search,q=searchQuery).items(maxTweets) :         

#         #Verify the tweet has place info before writing (It should, if it got past our place filter)
#         if tweet.place is not None:
            
#             #Write the JSON format to the text file, and add one to the number of tweets we've collected
#             f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
#             tweetCount += 1

#     #Display how many tweets we have collected
#     print("Downloaded {0} tweets".format(tweetCount))

Downloaded 2 tweets


In [66]:
# #Let's look at what we are actually printing to text  (if you don't know what JSON is: http://json.org/)
# tweet._json